In [1]:
# Ref.: https://github.com/arybressane/CEBD1260-BIG-DATA-ANALYTICS

In [2]:
# import the library
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# sklearn :: utils
from sklearn.model_selection import train_test_split

# sklearn :: models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# knn model
from sklearn.neighbors import KNeighborsRegressor

# decision tree model
from sklearn.tree import DecisionTreeRegressor

# sklearn :: evaluation metrics
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error


sns.set_style('whitegrid')

# Problem definition

Apply regression models to predict Airbnb prices

# Load the data

In [3]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')
print("df_train shape:"+str(df_train.shape) )
print("df_test shape:"+str(df_test.shape) )

df_train shape:(51000, 29)
df_test shape:(23111, 28)


In [4]:
print(df_train.columns)

Index(['id', 'log_price', 'property_type', 'room_type', 'amenities',
       'accommodates', 'bathrooms', 'bed_type', 'cancellation_policy',
       'cleaning_fee', 'city', 'description', 'first_review',
       'host_has_profile_pic', 'host_identity_verified', 'host_response_rate',
       'host_since', 'instant_bookable', 'last_review', 'latitude',
       'longitude', 'name', 'neighbourhood', 'number_of_reviews',
       'review_scores_rating', 'thumbnail_url', 'zipcode', 'bedrooms', 'beds'],
      dtype='object')


In [5]:
df_train.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,13662370,3.806662,House,Private room,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.5,Real Bed,strict,True,...,41.849684,-87.676270,Pilsen Arts Community Custom Home,Pilsen,17,97.0,https://a0.muscache.com/im/pictures/81318153/a...,60608,1.0,1.0
1,4765892,4.941642,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,2.0,Real Bed,strict,True,...,34.068613,-118.246455,Apartment 5 minutes from DTLA & Dodger Stadium,Echo Park,2,100.0,https://a0.muscache.com/im/pictures/aa00250e-0...,90012,1.0,1.0
2,21169968,4.941642,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,...,40.701958,-73.917352,"Brand New Huge 2bdr apartment(L,M train 2 min)",Bushwick,25,88.0,https://a0.muscache.com/im/pictures/d9220535-c...,11237,2.0,3.0
3,7939196,4.867534,Apartment,Entire home/apt,"{""Cable TV"",Internet,""Wireless Internet"",""Air ...",6,1.0,Real Bed,strict,True,...,40.742959,-73.990820,Grande Super Large APT !!!,Flatiron District,12,82.0,NaN,10010,1.0,3.0
4,18161036,3.663562,House,Private room,"{Internet,""Wireless Internet"",""Air conditionin...",2,1.0,Real Bed,flexible,True,...,34.046473,-117.734095,Private Cozy and Clean Rooms in Pomona,NaN,2,100.0,https://a0.muscache.com/im/pictures/e0c9b2f9-a...,91766,1.0,1.0


In [26]:
# print(df_train.isnull().sum())
# no blanks

# Feature Engineering 

#### Modify Columns names 

In [6]:
df_train[(df_train['zipcode'] == '95202\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\n94158')]

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
5890,18915873,4.65396,Apartment,Private room,"{Internet,""Wireless Internet"",""Air conditionin...",1,1.0,Real Bed,flexible,True,...,37.773742,-122.391503,San Francisco Luxury,Mission Bay,5,95.0,https://a0.muscache.com/im/pictures/13f8dcaa-e...,95202\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\n\r\r\r\r\...,1.0,1.0


In [13]:
df_train[(df_train['zipcode'] == ' ')]

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
50482,19864701,4.317488,Apartment,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",4,1.0,Real Bed,moderate,True,...,38.936838,-76.993548,Stylish Midrise Steps from Metro,NaN,25,88.0,https://a0.muscache.com/im/pictures/2f7dbb0b-a...,,1.0,1.0


In [14]:
print(df_train[(df_train['zipcode'] == '95202\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\n94158')]['zipcode'])
print(df_train[(df_train['zipcode'] == ' ')]['zipcode'])
# print(df_train[(df_train.id == 18915873)]['zipcode'])

#overwrite '95202\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\n\r\r\r\r\r\r\r\n94158'
# Ref.: https://www.dataquest.io/blog/settingwithcopywarning/
df_train.loc[df_train.id == 18915873,'zipcode'] = '95202_94158'
df_train.loc[df_train.id == 19864701,'zipcode'] = 'blank_over'

print(df_train[(df_train.id == 18915873)]['zipcode'])
print(df_train[(df_train.id == 19864701)]['zipcode'])

Series([], Name: zipcode, dtype: object)
50482     
Name: zipcode, dtype: object
5890    95202_94158
Name: zipcode, dtype: object
50482    blank_over
Name: zipcode, dtype: object


In [15]:
# df_train['zipcode'] = str(df_train['zipcode'])

In [16]:
# Ref.: https://github.com/arybressane/CEBD1260-BIG-DATA-ANALYTICS

# zipcode to numerical columns
df_zipcode = pd.get_dummies(df_train['zipcode'])
df = pd.concat([df_train, df_zipcode], axis=1).fillna(0.0)
print(list(df_zipcode.columns))
print('')
print(list(df.columns))


['02108', '02109', '02110', '02111', '02113', '02114', '02115', '02116', '02118', '02119', '02120', '02121', '02122', '02124', '02125', '02126', '02127', '02128', '02129', '02130', '02131', '02132', '02134', '02135', '02136', '02138', '02139', '02145', '02152', '02186', '02199', '02210', '02215', '02445', '02446', '02467', '07306', '10000', '10001', '10001.0', '10002', '10002.0', '10003', '10003-8623', '10003.0', '10004', '10004.0', '10005', '10006', '10007', '10009', '10009.0', '10010', '10010.0', '10011', '10011.0', '10012', '10012.0', '10013', '10013.0', '10014', '10014.0', '10016', '10017', '10018', '10018.0', '10019', '10019.0', '10021', '10022', '10023', '10024', '10025', '10026', '10026.0', '10027', '10027.0', '10028', '10029', '10029.0', '10030', '10031', '10032', '10033', '10034', '10035', '10035.0', '10036', '10036.0', '10037', '10037.0', '10038', '10038.0', '10039', '10040', '10044', '10048', '10065', '10069', '10075', '10106', '10118', '10119.0', '10128', '10128.0', '10129'

In [35]:
df_zipcode.dtypes

02108          uint8
02109          uint8
02110          uint8
02111          uint8
02113          uint8
02114          uint8
02115          uint8
02116          uint8
02118          uint8
02119          uint8
02120          uint8
02121          uint8
02122          uint8
02124          uint8
02125          uint8
02126          uint8
02127          uint8
02128          uint8
02129          uint8
02130          uint8
02131          uint8
02132          uint8
02134          uint8
02135          uint8
02136          uint8
02138          uint8
02139          uint8
02145          uint8
02152          uint8
02186          uint8
               ...  
94108          uint8
94109          uint8
94109.0        uint8
94110          uint8
94111          uint8
94112          uint8
94114          uint8
94114.0        uint8
94115          uint8
94115.0        uint8
94116          uint8
94117          uint8
94117.0        uint8
94118          uint8
94118.0        uint8
94121          uint8
94122        

In [17]:
df.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,...,94127,94129,94130,94131,94132,94133,94134,94158,95202_94158,blank_over
0,13662370,3.806662,House,Private room,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.5,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
1,4765892,4.941642,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,2.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
2,21169968,4.941642,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,...,0,0,0,0,0,0,0,0,0,0
3,7939196,4.867534,Apartment,Entire home/apt,"{""Cable TV"",Internet,""Wireless Internet"",""Air ...",6,1.0,Real Bed,strict,True,...,0,0,0,0,0,0,0,0,0,0
4,18161036,3.663562,House,Private room,"{Internet,""Wireless Internet"",""Air conditionin...",2,1.0,Real Bed,flexible,True,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Ref.: https://github.com/arybressane/CEBD1260-BIG-DATA-ANALYTICS
# select the columns
X_columns = ['accommodates', 'bathrooms', 'bedrooms', 'beds', 'review_scores_rating'] + list(df_zipcode.columns)
y_column = ['log_price']
print(X_columns)

['accommodates', 'bathrooms', 'bedrooms', 'beds', 'review_scores_rating', '02108', '02109', '02110', '02111', '02113', '02114', '02115', '02116', '02118', '02119', '02120', '02121', '02122', '02124', '02125', '02126', '02127', '02128', '02129', '02130', '02131', '02132', '02134', '02135', '02136', '02138', '02139', '02145', '02152', '02186', '02199', '02210', '02215', '02445', '02446', '02467', '07306', '10000', '10001', '10001.0', '10002', '10002.0', '10003', '10003-8623', '10003.0', '10004', '10004.0', '10005', '10006', '10007', '10009', '10009.0', '10010', '10010.0', '10011', '10011.0', '10012', '10012.0', '10013', '10013.0', '10014', '10014.0', '10016', '10017', '10018', '10018.0', '10019', '10019.0', '10021', '10022', '10023', '10024', '10025', '10026', '10026.0', '10027', '10027.0', '10028', '10029', '10029.0', '10030', '10031', '10032', '10033', '10034', '10035', '10035.0', '10036', '10036.0', '10037', '10037.0', '10038', '10038.0', '10039', '10040', '10044', '10048', '10065', '

In [19]:
# handle missing values
df_train = df_train[X_columns + y_column]
print(df_train.shape)

KeyError: "['02108' '02109' '02110' '02111' '02113' '02114' '02115' '02116' '02118'\n '02119' '02120' '02121' '02122' '02124' '02125' '02126' '02127' '02128'\n '02129' '02130' '02131' '02132' '02134' '02135' '02136' '02138' '02139'\n '02145' '02152' '02186' '02199' '02210' '02215' '02445' '02446' '02467'\n '07306' '10000' '10001' '10001.0' '10002' '10002.0' '10003' '10003-8623'\n '10003.0' '10004' '10004.0' '10005' '10006' '10007' '10009' '10009.0'\n '10010' '10010.0' '10011' '10011.0' '10012' '10012.0' '10013' '10013.0'\n '10014' '10014.0' '10016' '10017' '10018' '10018.0' '10019' '10019.0'\n '10021' '10022' '10023' '10024' '10025' '10026' '10026.0' '10027'\n '10027.0' '10028' '10029' '10029.0' '10030' '10031' '10032' '10033'\n '10034' '10035' '10035.0' '10036' '10036.0' '10037' '10037.0' '10038'\n '10038.0' '10039' '10040' '10044' '10048' '10065' '10069' '10075' '10106'\n '10118' '10119.0' '10128' '10128.0' '10129' '10129.0' '10162' '10270'\n '10280' '10281' '10282' '10282.0' '10301'

In [20]:
# probably not a good idea for 'review_scores_rating' as it fill empty reviews with 0 when the actual review score is unknown
df_train = df_train.fillna(0.0) 

print(df_train.shape)

(51000, 29)


# Model Training

In [ ]:
# split the data using sklearn

threshold = 0.8
X = df_train[X_columns]
y = df_train[y_column]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1.0-threshold, shuffle=True)

print('X_train', X_train.shape)
print('y_train', y_train.shape)
print('X_test', X_test.shape)
print('y_test', y_test.shape)

#### Linear regression 

In [ ]:
# train a linear regression


# Cross Validation

In [ ]:
# Ref.: https://github.com/arybressane/CEBD1260-BIG-DATA-ANALYTICS

In [ ]:
models = [
    ('LinearRegression', LinearRegression()),
    ('RandomForestRegressor10', RandomForestRegressor(n_estimators=10)),
    ('RandomForestRegressor100', RandomForestRegressor(n_estimators=100, n_jobs=4)),
    ('KNeighborsRegressor', KNeighborsRegressor()),
    ('DecisionTreeRegressor', DecisionTreeRegressor())
]

k = 10
results = {}
for m in models:
     print('MODEL', m[0])
     results[m[0]] = {'mae':[], 'rmse':[]}
     kf = KFold(n_splits=k)
     for train_index, test_index in kf.split(X):
         X_train_k, X_test_k = X.values[train_index], X.values[test_index]
         y_train_k, y_test_k = y.values[train_index], y.values[test_index]
         model = m[1]
         model.fit(X_train_k, y_train_k.ravel())
         y_pred = model.predict(X_test_k)
         mae = mean_absolute_error(y_test_k, y_pred)
         rmse = np.sqrt(mean_squared_error(y_test_k, y_pred))
         results[m[0]]['mae'].append(mae)
         results[m[0]]['rmse'].append(rmse)
            

In [ ]:
for metric in ['mae', 'rmse']:
     values = []
     labels = []
     for model, result_values in results.items():
         for m, v in result_values.items():
             if m == metric:
                 labels.append(model)
                 values.append(v)
     plt.figure(figsize=(12,6))
     plt.title(metric)
     plt.boxplot(values)
     plt.xticks(range(1, len(labels)+1), labels, rotation='horizontal')
     plt.show()
    

# Prepare submission

In [ ]:
df_prediction = df_test[X_columns].fillna(0.0)
df_test['log_price'] = model.predict(df_prediction)
df_test[['id', 'log_price']]

In [ ]:
df_test[['id', 'log_price']].to_csv('submission_linear_regression_modified.csv', index=False)